# Clustering Particional

## Machine Learning. Grado en Ciencia de datos

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics as metrics
from sklearn.preprocessing import StandardScaler
from scipy.cluster import vq
from sklearn.cluster import KMeans,DBSCAN

%matplotlib inline

### El conjunto de datos

Los datos sobre los que vamos a trabajar se basan en el conjunto de datos `protein`. Después de examinar los datos, vemos que los datos están separados por un tabulador y que los nombres de los países están en la primera columna. Para cargarlos debemos ejecutar la siguiente instrucción, en la que le indicamos el carácter de separación y que el nombre del país va a ser el índice del data frame.

In [17]:
proteindata = pd.read_csv('proteindata.txt', sep='\t', index_col=0)

Vamos a examinar los datos para ver si se han leido correctamente

In [ ]:
proteindata

Aparentemente, todos los datos han sido leídos correctamente. Sin embargo, nos queda por comprobar si los tipos asignados son los adecuados.

In [ ]:
proteindata.info()

Una vez comprobado que hemos leído correctamente los datos, tenemos que comprobar cómo están distribuidos los datos, para ver si es necesario una estandarización de los mismos. Para ello podemos realizar un gráfico de barras.

In [ ]:
proteindata.boxplot(grid=False, vert=False)

Como se puede apreciar es necesario que transformar los datos para que estos estén en la misma escala y los podamos comparar.

In [20]:
scaler = StandardScaler()
scaler.fit(proteindata)
data_scaled = scaler.transform(proteindata)

In [ ]:
protein_scaled = pd.DataFrame(data_scaled, columns=proteindata.columns, index=proteindata.index)
protein_scaled.boxplot(grid=False, vert=False)

### Kmeans

El módulo `sklearn.cluster` nos ofrece la función `KMeans()` que implementa la técnica de clustering de las **K medias**. Los parámetros más importantes de esta función son:
- **n_clusters (int)**: número de clusters objetivo.
- **init**: Método de inicialización de los centroides. Se puede utilizar **k-means++**, que es el valor por defecto o la inicialización aleatoria, **ramdom**. También se le puede pasar una matriz con las coordenadas de los centroides.
- **n_init (int)**: número de veces que se ejecuta el algoritmo con diferentes inicializaciones de los centroides. Se puede indiciar también **auto** y en ese caso se inicializará a 10 en el caso de que `init='random'` o 1 en el caso de `init='k-means++'`.
- **max_iter (int)**: número máximo de iteraciones del algoritmo para cada ejecución. También se le puede indicar la tolerancia, a través del parámetro **tol (float)**, a partir de la cual se considera que la diferencia de los centroides ha hecho converger al algoritmo.
- **random_state**: determina la semilla para la inicialización aleatoria de los centroides. Es importante utilizar este parámetro para garantizar la reproducibilidad.

A continuación se puede ver un ejemplo de como aplicar la técnica de las `K medias` al conjunto de datos de ejemplo.


In [ ]:
clusters_kmeans = KMeans(n_clusters = 4,random_state = 0,n_init=10)
clusters_kmeans.fit(protein_scaled)

Para poder ver la a qué cluster se ha asignado cada instancia sólo tenemos que acceder al atributo **labels_** del objeto generado.

In [ ]:
clusters_kmeans.labels_

De la misma forma, en el atributo **cluster_centers_** del mismo objeto tenemos la información sobre las coordenadas de los centroides, que pueden ser interesante para mostrar los resultados de forma gráfica.

In [ ]:
clusters_kmeans.cluster_centers_

A continuación, al igual que hicimos con el agrupamiento jerárquico, podemos ver los resultados del clustering utilizando un gráfico de dispersión al que le añadimos los centroides de cada cluster.

In [ ]:
fig, ax = plt.subplots()
centers_x = clusters_kmeans.cluster_centers_[:,protein_scaled.columns.get_loc('RedMeat')]
center_y = clusters_kmeans.cluster_centers_[:,protein_scaled.columns.get_loc('Fish')]
paises = np.array(protein_scaled.index).astype(str)
plt.scatter(protein_scaled["RedMeat"],protein_scaled["Fish"],
             c=clusters_kmeans.labels_, cmap='viridis')
for i, label in enumerate(paises):
    plt.annotate(label, (protein_scaled["RedMeat"][i], protein_scaled["Fish"][i]),
                 textcoords="offset points", xytext=(0, 5), ha='center', fontsize=6)
plt.scatter(centers_x,center_y,marker="*",c=[0,1,2,3],cmap='viridis')
plt.xlabel('Red Meat')
plt.ylabel('Fish')
plt.show()

También podemos usar la función **kmeans2()** del módulo `scipy.cluster.vq`. En este caso se utiliza el parámetro **k (int)** para indicar el número de clusters y se devuelven dos arrays uno con los centroides y otro con la asignación de instancias a los diferentes clusters. Aquí puedes ver un ejemplo de como utilizarla. En

In [13]:
centroids, labels = vq.kmeans2(protein_scaled,k=4,minit="++",seed=0,iter=10)

In [ ]:
centroids

In [ ]:
labels

### Kmedoids

La técnica de clustering de **K medoides** o **PAM (Partition Arround Medoids)** se puede aplicar a través de la función `KMedodis()` del módulo `sklearn_extra.cluster`. Los parámetros más importantes son:

- **n_clusters (int)**: número de clusters objetivo.
- **metric** métrica de distancia a utilizar (por defecto está inicializado a 'euclidean').
- **init**: Método de inicialización de los centroides. Se puede utilizar **k-medoids++**, que es el valor por defecto o la inicialización aleatoria, **ramdom**. También se le puede pasar una matriz con las coordenadas de los centroides. Otras opciones son: **heuristic**, que selecciona las *n_clusters* instancias con menor distancia total al resto de los elementos, o *build*, inicialización de medoides voraz definida en la descripción original del algoritmo.
- **n_init (int)**: número de veces que se ejecuta el algoritmo con diferentes inicializaciones de los centroides. Se puede indiciar también **auto** y en ese caso se inicializará a 10 en el caso de que `init='random'` o 1 en el caso de `init='k-means++'`.
- **max_iter (int)**: número máximo de iteraciones del algoritmo para cada ejecución. También se le puede indicar la tolerancia, a través del parámetro **tol (float)**, a partir de la cual se considera que la diferencia de los centroides ha hecho converger al algoritmo.
- **random_state**: determina la semilla para la inicialización aleatoria de los centroides. Es importante utilizar este parámetro para garantizar la reproducibilidad.

Para aplicar esta técnica  nuestro dataset hay que seguir los mismos pasos que indicamos en la aplicación de las *K medias*.

In [ ]:
from sklearn_extra.cluster import KMedoids

clusters_kmediods = KMedoids(n_clusters=4,random_state=0).fit(protein_scaled)

In [ ]:
clusters_kmediods.labels_

In [ ]:
clusters_kmediods.medoid_indices_

Otra librería interesante que implementa esta técnica es [kmedoids](https://python-kmedoids.readthedocs.io/en/latest/), que implementa versiones más eficientes de *K medoides*. Recuerda que para datasets grandes conviene utilizar la versión **Clara** del *K medoides*.

#### Ejercicio 1: Regla del codo

En este ejercicio vamos a desarrollar todos los pasos necesarios para aplicar la regla del codo. Recuerda que tenemos que calcular la distancia intracluster para determinados números de clusters y elegir aquel en el que se produzca un cambio de tendencia.

In [ ]:
## Define una array vacío, wss, para ir almacenando la distancia intracluster para cada número de clusters elegido.
## PON TU CÓDIGO AQUÍ


## Definir un bucle que vaya del número mínimo de clusters al número máximo de clusters.
## Para cada número de clusters aplicar el algoritmo k-medias.
## Una vez aplicado el algoritmo determinar la suma de distancia intraclusters y añadirla al array wss.
## PON TU CÓDIGO AQUÍ


## Una vez terminado el bucle crear un gráfico con los diferentes números de clusters en el eje X y la distancia intracluster en el eje Y.
## PON TU CÓDIGO AQUÍ




¿Cuál crees que puede ser el k óptimo?

#### Ejercicio 2. Selección del número de cluster óptimo utilizando el índice silueta.
Repite el ejercicio anterior pero dentro del bucle calcula el índice silueta medio. Recuerda que el **k** óptimo será el que tenga asociado el índice máximo

In [ ]:
# PON TU CÓIDIGO AQUÍ

### DBSCAN

El método de clustering **DBSCAN** lo podemos aplicar utilizando la función `DBSCAN()` del módulo `sklearn.cluster`. Los parámetros más importantes son:

- **eps (float)**: radio para la definición de la *epsilon-vecindad* (por defecto 0.5).
- **min_samples (int)**: mínimo número de puntos en la  *epsilon-vecindad* para definir un punto núcleo (por defecto 5).
- **metric**: métrica de distancia a utilizar, por defecto `metric='euclidean'`.

Los parámetros *eps* y *min_samples* son críticos para el funcionamiento del algoritmo. Para seleccionarlos de forma adecuada hay que analizar un gráfico en el que se muestren de forma ordenada de menor a mayor las distancias de cada punto a los `min_samples` instancias más cercanas. Esto se puede hacer utilizando la función  `NearestNeighbors()` del módulo `sklearn.neighbors` que nos devuelve una matriz de dimensiones $n\_samples \times n\_vecinos-1$ (la primera columna es la distancia de cada punto consigo mismo, la segunda es la distancia al segundo punto más cercano y, así, sucesivamente).

Vamos a construir el gráfico de las *KNN distancias*:

In [ ]:
from sklearn.neighbors import NearestNeighbors

# Llamamos a la función NearestNeighbors para calcular la distancia a los 4 vecinos más cercanos.

neighbors = NearestNeighbors(n_neighbors=4).fit(protein_scaled)

# Extraemos las distancias y los índices de los vecinos más cercanos con la función kneighbors().

distances, indices = neighbors.kneighbors(protein_scaled)

# Elimina la primera columna ya que contiene la distancia de cada punto consigo mismo
distances = distances[:,1:].reshape(-1)

# Transforma la matriz de distancia en una array de dimensión 1.
distances = distances.reshape(-1)

# Ordena las distancias en orden descendente
distances = np.sort(distances, axis=0)

# Crea un gráfico en el que el eje y represente las distancias.
plt.plot(distances)


Cómo se puede apreciar en el gráfico hay dos cambios de tendencia, uno muy evidente en torno a 3 y otro, más suave cercano a 2.5. Habría que probar estos valores teniendo en cuenta que min_samples lo hemos fijado a 3. Vamos a empezar con `eps=3.0`

In [ ]:
clusters_dbscan = DBSCAN(eps=3,min_samples=3).fit(protein_scaled)

Para determinar qué cluster ha sido asignado a cada instancia debemos acceder al atributo `labels_` del objeto generado.

In [ ]:
clusters_dbscan.labels_

Como podemos apreciar, se han asignado todos las instancias al mismo cluster, con lo que parece lógico que probemos con un valor localizado cercano a 2.5

In [ ]:
clusters_dbscan = DBSCAN(eps=2.3,min_samples=3).fit(protein_scaled)

In [ ]:
clusters_dbscan.labels_

#### Ejercicio 3.
1. ¿Qué crees que significa el cluster marcado con la etiqueta -1?
2. Construye un algoritmo para encontrar el eps óptimo utilizando el índice silueta.
